<a href="https://colab.research.google.com/github/WesleySantosMaxtelll/doutorado/blob/main/original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 14.5 MB/s 
     |████████████████████████████████| 596 kB 75.1 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 6.6 MB 69.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import collections
import matplotlib.pyplot as plt
import sys
import ast
import pandas as pd
import torch
import os
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
from torch.optim import AdamW, SGD
from transformers import BertTokenizerFast, BertModel, DistilBertTokenizer, DistilBertModel
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
from sklearn.utils import class_weight
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import _pickle as pickle
import time
from datetime import datetime, timedelta
import gc
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import random

In [ ]:
# path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/train_all_A_processed.pkl'
# X_processed, y = pickle.load(open(path, 'rb'))

In [ ]:
# import pickle
# from sklearn.metrics import classification_report

In [ ]:
# df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_depression.pkl', 'rb'))

In [ ]:
# print(classification_report(df['real'], df['pred']))

In [ ]:
import torch.nn as nn 
EMBED_DIMENSION = 2
EMBED_MAX_NORM = 1 
class CBOW_Model(nn.Module):
    def __init__(self, vocab_size: int):
        super(CBOW_Model, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=EMBED_DIMENSION,
            max_norm=EMBED_MAX_NORM,
        )
        self.linear = nn.Linear(
            in_features=EMBED_DIMENSION,
            out_features=vocab_size,
        )
    def forward(self, inputs_):
        x = self.embeddings(inputs_)
        # return x
        x = x.mean(axis=1)
        x = self.linear(x)
        return x

In [ ]:

def load_data(portion, task, sel):
    df_c = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{portion}_{task}_c_SetembroBR_v6.csv', sep=';')
    df_d = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{portion}_{task}_SetembroBR_v6.csv', sep=';')
    df_c_net = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{task}_ctrl-users-network_final_v6.csv', sep=';')
    df_d_net = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{task}_diag-users-network_final_v6.csv', sep=';')
    if sel not in ['all', 'balanced', 'medical_terms', 'pref_medical_terms', 'r80', 'dev80']:  
        df_d = df_d[df_d['User_ID'].isin(df_d_net[df_d_net['Event_Type'] == sel]['User_ID'])]
        df_c_net = df_c_net[df_c_net['Counterpart'].isin(df_d['User_ID'])]
        df_c = df_c[df_c['User_ID'].isin(df_c_net['User_ID'])]
    if sel == 'balanced' and portion == 'train':
        # return df_c, df_d, df_c_net
        dt = {'D':'depression', 'A': 'anxiety'}
        user_id_c_all = []
        for _, user_id in df_d.iterrows():
            user_id_c = df_c_net[df_c_net['Counterpart'] == user_id['User_ID']].sample(n=1).iloc[0]['User_ID']
            user_id_c_all.append(user_id_c)
        df_c = df_c[df_c['User_ID'].isin(set(user_id_c_all))] 
    return pd.concat((df_c, df_d))


In [ ]:
# df_c = load_data('train', 'D', 'balanced')

In [ ]:
# df_c[df_c['Diagnosed_YN'] != 'yes']

In [ ]:
import re

# ao copiar/colar verifique se os caracteres de acentuaçao nas regexs estão corretos
def makeTags(text):
    text = str(text).lower()
    if re.search(r"diagn[oó]stic", text, flags=re.DOTALL):
        return 'diagnostico'
    if re.search(r"tratamento", text, flags=re.DOTALL):
        return 'tratamento'
    if re.search(r"depress[aã]o", text, flags=re.DOTALL):
        return 'depressao'
    if re.search(r"ansiedade", text, flags=re.DOTALL):
            return 'ansiedade'
    if re.search(r"depressivo|tarja\sp|ansiol[íi]tico", text, flags=re.DOTALL):
        return 'antidepress'
    if re.search(r"m[eé]dic[oa]|psic[oó]l[oó]g[oa]|psiquiatra|neurologista|terapeuta", text, flags=re.DOTALL):
        return 'medico'
    return ''

# exemplo de uso - marca todos os textos que contêm palavras de natureza clínica
def filter_medical_terms(texts):
    sel_texts_ = []
    for text_list in tqdm(texts):
        sel_texts = [tt for tt in text_list if makeTags(tt) == '']
        sel_texts_.append(sel_texts)
    return sel_texts_


# exemplo de uso - marca todos os textos que contêm palavras de natureza clínica
def sel_medical_terms(texts):
    size = 100
    sel_texts_ = []
    for text_list in tqdm(texts):
        sel_texts = [tt for tt in text_list if makeTags(tt) == '']
        med_sel_texts = [tt for tt in text_list if makeTags(tt) != '']
        if size <= len(med_sel_texts):
            sel_texts_.append(med_sel_texts[-size:])
        else:
            l = size - len(med_sel_texts)
            # print(l)
            if len(sel_texts) > l:
                add_texts = random.sample(sel_texts, l)
                med_sel_texts = med_sel_texts + add_texts
                random.shuffle(med_sel_texts)
                sel_texts_.append(med_sel_texts)
            else:
                sel_texts_.append(med_sel_texts + sel_texts)

    return sel_texts_
  
def random_cont_selection(texts, n=80):
    sel_texts_ = []
    for text_list in tqdm(texts):
        fp = random.randint(n, len(text_list))
        sp = fp - n
        sel_texts_.append(text_list[sp:fp])

    return sel_texts_

def recent_selection(texts, n=80):
    sel_texts_ = []
    for text_list in tqdm(texts):
        sel_texts_.append(text_list[-n:])

    return sel_texts_


In [ ]:
def load_convert(task, sel):
    df_test = load_data('test', task, sel)
    df_train = load_data('train', task, sel)
    df_test['Text'] = df_test['Text'].apply(lambda x: x.split('#'))
    df_test['label'] = df_test['Diagnosed_YN'].apply(lambda x: 1 if x == 'yes' else 0)
    df_train['Text'] = df_train['Text'].apply(lambda x: x.split('#'))
    df_train['label'] = df_train['Diagnosed_YN'].apply(lambda x: 1 if x == 'yes' else 0)
    train_initial_size = df_train['Text'].apply(lambda x: len(x)).sum()
    test_initial_size = df_test['Text'].apply(lambda x: len(x)).sum()
    random.seed(42)
    if sel == 'medical_terms': 
        print('Start filter')
        df_train['Text'] = filter_medical_terms(df_train['Text'])
        df_test['Text'] = filter_medical_terms(df_test['Text'])
        train_final_size = df_train['Text'].apply(lambda x: len(x)).sum()
        test_final_size = df_test['Text'].apply(lambda x: len(x)).sum()
        print(f'On task {task}.\n\tThe number of samples in the train dataset is ')
    elif sel == 'pref_medical_terms': 
        print('sel', sel)
        df_train['Text'] = sel_medical_terms(df_train['Text'])
        df_test['Text'] = sel_medical_terms(df_test['Text'])
        train_final_size = df_train['Text'].apply(lambda x: len(x)).sum()
        test_final_size = df_test['Text'].apply(lambda x: len(x)).sum()
        print(f'On task {task}.\n\tThe number of samples in the train dataset is ', len(df_train))
    elif sel == 'r80':
        print('sel', sel)
        df_train['Text'] = random_cont_selection(df_train['Text'])
        # df_test['Text'] = random_cont_selection(df_test['Text'])
    elif sel == 'dev80':
        print('sel', sel)
        df_train['Text'] = recent_selection(df_train['Text'])
        # df_test['Text'] = random_cont_selection(df_test['Text'])
    return df_train, df_test

In [ ]:
# df_train, df_test = load_convert('D',  'all')
# print('Depression all tweets train', sum(df_train['Text'].apply(lambda x: len(x))))
# print('Depression all tweets test', sum(df_test['Text'].apply(lambda x: len(x))))
# df_train, df_test = load_convert('A',  'all')
# print('Anxiety all tweets train', sum(df_train['Text'].apply(lambda x: len(x))))
# print('Anxiety all tweets test', sum(df_test['Text'].apply(lambda x: len(x))))
# df_train, df_test = load_convert('D',  'medical_terms')
# print('Depression medical_terms tweets train', sum(df_train['Text'].apply(lambda x: len(x))))
# print('Depression medical_terms tweets test', sum(df_test['Text'].apply(lambda x: len(x))))
# df_train, df_test = load_convert('A',  'medical_terms')
# print('Anxiety medical_terms tweets train', sum(df_train['Text'].apply(lambda x: len(x))))
# print('Anxiety medical_terms tweets test', sum(df_test['Text'].apply(lambda x: len(x))))

In [ ]:
# sel_medical_terms(df_test['Text'].iloc[:2])

In [ ]:
# sum(df[0]['Text'].apply(lambda x: len(x)))

In [ ]:
import random

class CustomTextDataset(Dataset):
    def __init__(self, tokens_mask, labels):
        self.tokens_mask = tokens_mask
        self.labels = labels
        self.size = 10
        random.seed(42)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        data = self.tokens_mask[idx]
        start = random.randint(0, data['tokens'].shape[0] - self.size)
        data_short = {v: data[v][start: start + self.size] for v in data}
        return [data_short, label]


In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda')
print(device)
# from keras.preprocessing.sequence import pad_sequences
source_folder = 'Data'
destination_folder = 'Model'


# Define the function for creating a weight dictionary.
def create_weight_dict(labels):
    unique_labels = np.unique(labels)
    class_weights = class_weight.compute_class_weight('balanced', classes=unique_labels, y=labels)
    return class_weights


def get_loss(outputs, b_labels, vect_weights):
    loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32))
    loss_fct.to(device)
    loss = loss_fct(outputs[1].view(-1, 2), b_labels.view(-1))

    return loss

cuda


In [ ]:
class CNN_Yates(nn.Module):
    def __init__(self, device, output_size=2):
        super(BertUser, self).__init__()
        self.device = device
        pre_trained = 'neuralmind/bert-base-portuguese-cased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.hidden_size = self.bert.config.hidden_size
        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=100,
                           num_layers=4, bidirectional=True)
        self.sample_size = 10
        self.fc_out3 = nn.Linear(6000, 1000)
        self.fc_out2 = nn.Linear(1000, 100)
        self.fc_out1 = nn.Linear(100 * self.sample_size, output_size)

    def forward(self, tokens, masks):
        outputs = []
        # iterate in the batch through all sequences
        for token, mask in zip(tokens, masks):
            encoded = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            out, hiden = self.rnn(encoded)
            out = self.fc_out3(out.view(self.sample_size, -1))
            out = F.dropout(out, 0.1)
            out = self.fc_out2(out)
            out = F.dropout(out, 0.1)
            out = self.fc_out1(out.view(-1))
            out = torch.softmax(out, -1)
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)
        return outputs



class BertUser(nn.Module):
    def __init__(self, device, output_size=2):
        super(BertUser, self).__init__()
        self.device = device
        pre_trained = 'neuralmind/bert-base-portuguese-cased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.hidden_size = self.bert.config.hidden_size
        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=100,
                           num_layers=4, bidirectional=True)
        self.sample_size = 10
        self.fc_out3 = nn.Linear(6000, 1000)
        self.fc_out2 = nn.Linear(1000, 100)
        self.fc_out1 = nn.Linear(100 * self.sample_size, output_size)

    def forward(self, tokens, masks):
        outputs = []
        # iterate in the batch through all sequences
        for token, mask in zip(tokens, masks):
            encoded = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            out, hiden = self.rnn(encoded)
            out = self.fc_out3(out.view(self.sample_size, -1))
            out = F.dropout(out, 0.1)
            out = self.fc_out2(out)
            out = F.dropout(out, 0.1)
            out = self.fc_out1(out.view(-1))
            out = torch.softmax(out, -1)
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)
        return outputs

class ExpertGate(nn.Module):
    def __init__(self, device, s, pre_trained='Geotrend/distilbert-base-pt-cased', output_size=2):
        super(ExpertGate, self).__init__()
        self.device = device
        self.hidden_size = s
        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=100,
                           num_layers=4, bidirectional=True)
        self.sample_size = 10
        self.fc_out3 = nn.Linear(6000, 1000)
        self.fc_out2 = nn.Linear(1000, 100)
        self.fc_out1 = nn.Linear(100 * self.sample_size, output_size)

    def forward(self, bert_vectors):
        outputs = []
        # iterate in the batch through all sequences
        for encoded in bert_vectors:
            # encoded = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            out, hiden = self.rnn(encoded)
            out = self.fc_out3(out.view(self.sample_size, -1))
            out = F.dropout(out, 0.5)
            out = self.fc_out2(out)
            out = F.dropout(out, 0.5)
            out = self.fc_out1(out.view(-1))
            out = torch.softmax(out, -1)
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)
        return outputs

class MoE(nn.Module):
    def __init__(self, device, exps=3):
        super(MoE, self).__init__()
        pre_trained = 'neuralmind/bert-base-portuguese-cased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.exps = exps
        self.device = device
        self.nets = nn.ModuleList([
          ExpertGate(device, self.bert.config.hidden_size) for _ in range(self.exps)
        ])
        self.gating = ExpertGate(device, self.bert.config.hidden_size, output_size=exps)
    
    def forward(self, atts, masks):
        bert_out = []
        for token, mask in zip(atts, masks):
            # print(1)
            bert_vectors = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            bert_out.append(bert_vectors)
        # print(2)
        weights_sof = self.gating(bert_out)
        # print(3)
        preds = []
        for n in self.nets:
            # print(4)
            preds.append(n(bert_out))
        # print(5)
        return (torch.stack(preds) * weights_sof.T[..., None]).sum(0)
    
class Ensemble(nn.Module):
    def __init__(self, device):
        super().__init__()
        pre_trained = 'neuralmind/bert-base-portuguese-cased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.nets = nn.ModuleList()
        for _ in range(3):
            self.nets.append(ExpertGate(device, self.bert.config.hidden_size))
        # self.gate = MixedBertModel(4)

    def forward(self, atts, masks):
        bert_out = []
        for token, mask in zip(atts, masks):
            # print(1)
            bert_vectors = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            bert_out.append(bert_vectors)
        
        preds = []
        for n in self.nets:
            # print(4)
            preds.append(n(bert_out))

        with torch.no_grad():

            ens_outs = torch.stack(preds).mean(0)
        # gate_output = self.gate(input_ids=inputs[0], attention_mask=inputs[1])

        # ensemble output and the models output
        return ens_outs, preds

class EnsembleConcat(nn.Module):
    def __init__(self, device):
        super().__init__()
        pre_trained = 'neuralmind/bert-base-portuguese-cased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.nets = nn.ModuleList()
        for _ in range(3):
            self.nets.append(ExpertGate(device, self.bert.config.hidden_size))
        # self.gate = MixedBertModel(4)

    def forward(self, atts, masks):
        bert_out = []
        for token, mask in zip(atts, masks):
            # print(1)
            bert_vectors = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            bert_out.append(bert_vectors)
        
        preds = []
        for n in self.nets:
            # print(4)
            preds.append(n(bert_out))

        # with torch.no_grad():

        ens_outs = torch.stack(preds).mean(0)
        # gate_output = self.gate(input_ids=inputs[0], attention_mask=inputs[1])

        # ensemble output and the models output
        return ens_outs

In [ ]:
from tqdm import tqdm

def get_tokens_mask_dates_split(X=None, y=None, tokenizer=None, type_data=None, task=None, sel=None, ml=30):
    path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{type_data}_{sel}_{task}_processed.pkl'

    if os.path.isfile(path):
        print('load', path)
        # 0/0
        X_processed, y = pickle.load(open(path, 'rb'))
    else:
        print('Prepare')
        X_processed = []
        for user_posts in tqdm(X):
            # print(tokenizer)
            user_posts = user_posts[-1500:]
            tokenized_texts = tokenizer.batch_encode_plus(user_posts, max_length=ml, padding='max_length', truncation=True)
            tokens_tensor = torch.tensor(tokenized_texts['input_ids'])
            segments_tensors = torch.tensor(tokenized_texts['attention_mask'])
            X_processed.append(
                {'tokens': tokens_tensor,
                 'mask': segments_tensors,
                 }
            )
        pickle.dump([X_processed, y], open(path, 'wb'))
    return X_processed, y
  

In [ ]:
def save_checkpoint(save_path, model, valid_loss):
    if save_path == None:
        return

    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

In [ ]:
def get_dataloader(X_, y_, shuffle):
    batch_size = 16
    TD_ = CustomTextDataset(X_, y_)
    return DataLoader(TD_, batch_size=batch_size, shuffle=shuffle, collate_fn=(lambda x: x))


In [ ]:
def load_checkpoint(load_path, model):
    if load_path == None:
        return

    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [ ]:
def loss_update(weights, ens_out, nets_out, y, optimizer):
    print(y, ens_out)
    batch_size = len(y)
    nb_digits = 2
    y_onehot = torch.FloatTensor(batch_size, nb_digits).to(device)

    # # In your for loop
    y_onehot.zero_()
    y_onehot.scatter_(1, y.view(-1, 1), 1)
    losses = []
    for i in range(len(nets_out)):
        net_out = nets_out[i]
        js = []
        for j in range(len(nets_out)):
            if i != j:
                netj_out = nets_out[j]
                js.append(netj_out - ens_out)
        ens_loss_i = ((net_out - ens_out) * torch.stack(js).sum(0)).mean(0)
        # weights * ((y_onehot - net_out) ** 2).mean(0) / 2
        # return weights, y_onehot,  net_out, ens_loss_i
        # print(((y_onehot - net_out) ** 2))
        l = (torch.tensor(weights).to(device) * ((y_onehot - net_out) ** 2).mean(0)/2 + ens_loss_i * 0.8).mean()
        losses.append(l)
    # print(losses)
    torch.stack(losses).mean(0).backward()
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
from sklearn.model_selection import train_test_split


def run_train(task, sel, model_name):
    print(task, sel, model_name)
    try:
        print('Load')
        # 1/0
        X_train, y_train = get_tokens_mask_dates_split(None, None, None, 'train', task, sel)
        X_test, y_test = get_tokens_mask_dates_split(None, None, None, 'test', task, sel)    
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    except:
        print('Create')
        df_train, df_test = load_convert(task, sel)
        print(len(df_train), len(df_test))   
        # return df_train, df_test
        tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
        # tokenizer = DistilBertTokenizer.from_pretrained('Geotrend/distilbert-base-pt-cased')
        X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, sel)
        X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, sel)    
        # return X_train, y_train
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    dataloader_train = get_dataloader(X_train, y_train, True)
    dataloader_val = get_dataloader(X_val, y_val, True)
    dataloader_test = get_dataloader(X_test, y_test, False)
    # return
    device = 'cuda'
    if model_name == 'lstm':
        model = BertUser(device)
    elif model_name == 'moe':
        model = MoE(device)
    elif model_name == 'ens':
        model = Ensemble(device)
    elif model_name == 'ens-concat':
        model = EnsembleConcat(device)
    else:
        raise Exception('Model not defined')

    model.to(device)

    # file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/model-{model_name}-{task}-sel-{sel}-epoch15.pt'    
    print('*' * 20)
    # load_checkpoint(file_path, model)
    
    lr = 1e-5
    optimizer = AdamW(model.parameters(), lr=lr)

    train_loss_set = []
    epochs = 30
    file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/'
    vect_weights = create_weight_dict(y_train)
    print(vect_weights)
    loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32), reduction='mean')
    loss_fct.to(device)
    for epoch in range(epochs):
        print(f'Epoch {epoch}')
        model.train()
        pred, true_label = [], []
        print(len(dataloader_train))
        for i, batch in enumerate(dataloader_train):
            if i % 100 == 0:
                print(datetime.fromtimestamp(time.time()), i, train_loss_set[-1] if len(train_loss_set) else 0)
            atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
            masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)

            if model_name == 'ens':
                # print('predict')
                outputs, model_outputs = model(atts, masks)
                # print('update weights')
                loss_update(vect_weights, outputs, model_outputs, torch.tensor([b[1] for b in batch]).to(device), optimizer)
                # decorrelation_ensemble_error(torch.tensor([b[1] for b in batch]).to(device), model_outputs, vect_weights, optimizer, num_experts=3, num_classes=2) # alternate is True!!
            else:
                outputs = model(atts, masks)
                loss = loss_fct(outputs, torch.tensor([b[1] for b in batch]).to(device))
                # output = loss(torch.argmax(outputs[1], dim=1).add_(-1), b_labels.add_(-1))
                train_loss_set.append(loss.item())
                # Backward pass
                loss.backward()
                # Update parameters and take a step using the computed gradient
                optimizer.step()
                # scheduler.step()
                optimizer.zero_grad()
            
            pred = pred + outputs.cpu().argmax(1).tolist()
            true_label = true_label + [b[1] for b in batch]
            gc.collect()
            torch.cuda.empty_cache()
        print('Results')
        print(classification_report(true_label, pred))
        # continue
        model.eval()
        with torch.no_grad():
            # validation loop
            pred, real = [], []
            for j, batch in enumerate(dataloader_val):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                if model_name == 'ens':
                    outputs, model_outputs = model(atts, masks)
                else:
                    outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            print(classification_report(real, pred))
            save_checkpoint(file_path + '/' + f'model-{model_name}-{task}-sel-{sel}-epoch{epoch}.pt', model, 0)
            # save_metrics(file_path + '/' + f'metrics-full-{task}-epoch{epoch}.pt', 0,0,0)
        model.train()



In [ ]:
import itertools

tasks = ['A']
selection = ['balanced']
models = ['lstm']

for task, sel, model_name in itertools.product(tasks, selection, models):
    run_train(task, sel, model_name)

A balanced lstm
Load
Prepare


0it [00:00, ?it/s]

Create


3550 3552


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


********************
[1.02379236 0.97728837]
Epoch 0
178
2022-07-03 03:15:09.778649 0 0
2022-07-03 03:17:38.585011 100 0.6916353106498718
Results
              precision    recall  f1-score   support

           0       0.46      0.36      0.40      1387
           1       0.49      0.59      0.54      1453

    accuracy                           0.48      2840
   macro avg       0.48      0.48      0.47      2840
weighted avg       0.48      0.48      0.47      2840



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       388
           1       0.45      1.00      0.62       322

    accuracy                           0.45       710
   macro avg       0.23      0.50      0.31       710
weighted avg       0.21      0.45      0.28       710

Model saved to ==> /content/drive/MyDrive/doutorado/diag_trat_saving/balanced//model-lstm-A-sel-balanced-epoch0.pt
Epoch 1
178
2022-07-03 03:19:45.404325 0 0.6928070187568665
2022-07-03 03:22:13.440054 100 0.6900238394737244
Results
              precision    recall  f1-score   support

           0       0.54      0.46      0.50      1387
           1       0.55      0.63      0.59      1453

    accuracy                           0.55      2840
   macro avg       0.55      0.55      0.54      2840
weighted avg       0.55      0.55      0.54      2840

              precision    recall  f1-score   support

           0       0.70      0.27      0.39       388
    

In [ ]:
df_train, df_test = load_convert(task, sel)

sel pref_medical_terms


0it [00:00, ?it/s]
0it [00:00, ?it/s]

On task A.
	The number of samples in the train dataset is  0


In [ ]:
df_test = load_data('test', task, sel)

In [ ]:
df_test

,User_ID,Diagnosed_YN,Text
0,CA_14150,no,@ Gente ressentida e invejosa! # @ nem deu raz...
1,CA_14381,no,"Última semana de férias, o tristeza :( # Que v..."
2,CA_14475,no,Momentos bons e ruins fazem parte da vida . A ...
3,CA_14701,no,"daqui a pouco é meu niver, já falei que odeio ..."
4,CA_14741,no,G.U.Y nos mundiais <3 # sobre a Luciana Genro ...
...,...,...,...
439,A_1213,yes,"Nunca façam tudo por alguém, você nunca vai se..."
440,A_700,yes,"@ vamos, pode começar # @ sai demoin, isso não..."
441,A_1785,yes,Menos aquele garoto lá # Mas agora tenho que v...
442,A_1794,yes,Os guris só falam bobagem kkk pqp # 0 da man...


In [ ]:
import re 
import gc
import itertools

def make_prediction(model, dataloader_, model_name):
    model.eval()
    with torch.no_grad():
        # validation loop
        all_pred = []
        for _ in range(5):
            pred, real = [], []
            for j, batch in enumerate(dataloader_):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                
                if model_name == 'ens':
                    outputs, model_outputs = model(atts, masks)
                else:
                    outputs = model(atts, masks)
                # outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            print(classification_report(real, pred))
            all_pred.append(pred)

        p = (np.array(all_pred).mean(0) > 0.5).astype(int)
        # pred, real = [], []
        # for j, batch in enumerate(dataloader_):
        #     atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
        #     masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
        #     if model_name == 'ens':
        #         outputs, model_outputs = model(atts, masks)
        #     else:
        #         outputs = model(atts, masks)

        #     pred = pred + outputs.cpu().argmax(1).tolist()
        #     real = real + [b[1] for b in batch]
        print("*" * 20, '\nALL')
        print(classification_report(real, p))
        print("*" * 20)


def evaluate_results(task, sel, model_name):
    gc.collect()
    device = 'cuda'
    if model_name == 'lstm':
        model = BertUser(device)
    elif model_name == 'moe':
        model = MoE(device)
    elif model_name == 'ens':
        model = Ensemble(device)
    elif model_name == 'ens-concat':
        model = EnsembleConcat(device)
    else:
        raise Exception('Model not defined')

    model.to(device)
    file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/'
    files = os.listdir(file_path)
    regex = re.compile(f'model-{model_name}-{task}-sel-{sel}-epoch')
    files = [f for f in files if regex.search(f)]
    df_train, df_test = load_convert(task, sel)    
    tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
    X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, sel)
    # X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, sel)    
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    del X_train
    del y_train
    del df_test
    gc.collect()
    dataloader_val = get_dataloader(X_val, y_val, True)
    # dataloader_test = get_dataloader(X_test, y_test, False)
    
    for f in files:
        print('*' * 20, f'\n{f}')
        try:
            load_checkpoint(f'{file_path}{f}', model)
        except:
            continue
        model.eval()
        make_prediction(model, dataloader_val, model_name)
    del df_train
    del X_val
    del y_val
    del dataloader_val
    gc.collect()



tasks = ['A']
selection = ['all']
models = ['ens-concat']

for task, sel, model_name in itertools.product(tasks, selection, models):
    # print(task, sel, model_name)
    evaluate_results(task, sel, model_name)

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

load /content/drive/MyDrive/doutorado/diag_trat_saving/train_all_A_processed.pkl
******************** 
model-ens-concat-A-sel-all-epoch0.pt
Model loaded from <== /content/drive/MyDrive/doutorado/diag_trat_saving/all/model-ens-concat-A-sel-all-epoch0.pt
              precision    recall  f1-score   support

           0       0.90      0.72      0.80      2515
           1       0.15      0.38      0.22       325

    accuracy                           0.68      2840
   macro avg       0.53      0.55      0.51      2840
weighted avg       0.82      0.68      0.73      2840

              precision    recall  f1-score   support

           0       0.90      0.72      0.80      2515
           1       0.16      0.40      0.23       325

    accuracy                           0.68      2840
   macro avg       0.53      0.56      0.51      2840
weighted avg       0.82      0.68      0.74      2840

              precision    recall  f1-score   support

           0       0.89      0.72     

In [ ]:
best = [
  'A lstm diagnosis 12',
  'A moe diagnosis 19',
  'A ens-concat diagnosis 23',
  'A lstm treatment 18',
  'A moe treatment 29',
  'A ens-concat treatment 12',
  'D lstm diagnosis 18',
  'D moe diagnosis 16',
  'D ens-concat diagnosis 22',
  'D lstm treatment 13',
  'D moe treatment 17',
  'D ens-concat treatment 12',
  'A lstm all 18',
  'A moe all 14',
  'A ens-concat all 20',
  'D lstm all 16',
  'D moe all 19',
  'D ens-concat all 18',
  'A lstm medical_terms 20',
  'D lstm medical_terms 21',
]

best = [
  'A lstm all 27',
  'D lstm all 25',
  'D moe all 17',
  'D ens-concat all 11',
  'A moe all 12',
  'A ens-concat all 15',
]
best = [
  'A lstm medical_terms 20',
  'D lstm medical_terms 23'
]
best = [
  'A lstm pref_medical_terms 9',
  'D lstm pref_medical_terms 4'
]

best = [
  'A lstm r80 14',
  'D lstm r80 10'
]

best = [
  'A lstm all 27',
  'D lstm all 25'
]
# best = [
#    'A moe all 3'
#  ]

# best = [
#    'D ens-concat all 24',
#    'A ens-concat all 19',
#  ]

# best = [
#    'D lstm balanced 21',
#    'A lstm balanced 26',
#  ]
# best = [
#   'A lstm dev80 23',
#   'D lstm dev80 8'
# ]



best = [
  'A lstm diagnosis 12',
  'A lstm treatment 18',
  'D lstm diagnosis 18',
  'D lstm treatment 13',
]

In [ ]:
def print_size(task, sel):
    df_train, df_test = load_convert(task, sel)  
    print(task, sel)
    print('Train size')
    print(Counter(df_train['label']))
    print('Test size')
    print(Counter(df_test['label']))

for task in ['A', 'D']:
    for sel in ['treatment', 'diagnosis']:
        print_size(task, sel)


A treatment
Train size
Counter({0: 4886, 1: 698})
Test size
Counter({0: 1071, 1: 153})
A diagnosis
Train size
Counter({0: 7539, 1: 1077})
Test size
Counter({0: 2037, 1: 291})
D treatment
Train size
Counter({0: 3031, 1: 433})
Test size
Counter({0: 798, 1: 114})
D diagnosis
Train size
Counter({0: 6398, 1: 914})
Test size
Counter({0: 1561, 1: 223})


In [ ]:

def test_results(task, model_name, sel, epoch):
    device = 'cuda'
    if model_name == 'lstm':
        model = BertUser(device)
    elif model_name == 'moe':
        model = MoE(device)
    elif model_name == 'ens':
        model = Ensemble(device)
    elif model_name == 'ens-concat':
        model = EnsembleConcat(device)
    else:
        raise Exception('Model not defined')

    model.to(device)
    file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/model-{model_name}-{task}-sel-{sel}-epoch{epoch}.pt'    
    print('*' * 20)
    load_checkpoint(file_path, model)
    model.eval()
    df_train, df_test = load_convert(task, sel)
    # return df_train, df_test 
    tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
    # X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, sel)
    X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, sel)    
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    dataloader_test = get_dataloader(X_test, y_test, False)
    with torch.no_grad():
        # validation loop
        all_pred = []
        for _ in range(50):
            pred, real = [], []
            for j, batch in enumerate(dataloader_test):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                
                if model_name == 'ens':
                    outputs, model_outputs = model(atts, masks)
                else:
                    outputs = model(atts, masks)
                # outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            # return real, pred
            print(classification_report(real, pred))
            all_pred.append(pred)

    p = (np.array(all_pred).mean(0) > 0.5).astype(int)
    d = {
        'pred': p,
         'pred_all': np.array(all_pred).mean(0),
         'real': real
    }
    df = pd.DataFrame(d)
    d_p = {'A': 'anxiety', 'D': 'depression'}
    pickle.dump(df, open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/{model_name}_{sel}_{d_p[task]}.pkl', 'wb'))
    print(classification_report(real, p))


for t in best:
    task, model_name, sel, epoch = t.split()
    print(task, model_name, sel, epoch)
    test_results(task, model_name, sel, epoch)
    # break

A lstm diagnosis 12


Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


********************
Model loaded from <== /content/drive/MyDrive/doutorado/diag_trat_saving/diagnosis/model-lstm-A-sel-diagnosis-epoch12.pt


...

********************
Model loaded from <== /content/drive/MyDrive/doutorado/diag_trat_saving/treatment/model-lstm-A-sel-treatment-epoch18.pt
load /content/drive/MyDrive/doutorado/diag_trat_saving/test_treatment_A_processed.pkl
              precision    recall  f1-score   support

           0       0.92      0.75      0.83      1071
           1       0.24      0.55      0.33       153

    accuracy                           0.72      1224
   macro avg       0.58      0.65      0.58      1224
weighted avg       0.84      0.72      0.76      1224

              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1071
           1       0.20      0.48      0.29       153

    accuracy                           0.70      1224
   macro avg       0.56      0.61      0.55      1224
weighted avg       0.82      0.70      0.74      1224

              precision    recall  f1-score   support

           0       0.92      0.74      0.82      1071
          

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


********************
Model loaded from <== /content/drive/MyDrive/doutorado/diag_trat_saving/diagnosis/model-lstm-D-sel-diagnosis-epoch18.pt
load /content/drive/MyDrive/doutorado/diag_trat_saving/test_diagnosis_D_processed.pkl
              precision    recall  f1-score   support

           0       0.93      0.70      0.80      1561
           1       0.23      0.60      0.33       223

    accuracy                           0.69      1784
   macro avg       0.58      0.65      0.56      1784
weighted avg       0.84      0.69      0.74      1784

              precision    recall  f1-score   support

           0       0.92      0.72      0.80      1561
           1       0.21      0.53      0.30       223

    accuracy                           0.69      1784
   macro avg       0.56      0.63      0.55      1784
weighted avg       0.83      0.69      0.74      1784

              precision    recall  f1-score   support

           0       0.92      0.73      0.82      1561
          

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


********************
Model loaded from <== /content/drive/MyDrive/doutorado/diag_trat_saving/treatment/model-lstm-D-sel-treatment-epoch13.pt
load /content/drive/MyDrive/doutorado/diag_trat_saving/test_treatment_D_processed.pkl
              precision    recall  f1-score   support

           0       0.90      0.86      0.88       798
           1       0.24      0.31      0.27       114

    accuracy                           0.79       912
   macro avg       0.57      0.59      0.58       912
weighted avg       0.82      0.79      0.80       912

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       798
           1       0.22      0.26      0.24       114

    accuracy                           0.79       912
   macro avg       0.56      0.56      0.56       912
weighted avg       0.81      0.79      0.80       912

              precision    recall  f1-score   support

           0       0.89      0.86      0.87       798
          

In [ ]:
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_depression.pkl', 'rb'))

In [ ]:
# depression BERT
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_all_depression.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.93      0.80      0.86      2359
           1       0.30      0.60      0.40       337

    accuracy                           0.77      2696
   macro avg       0.61      0.70      0.63      2696
weighted avg       0.85      0.77      0.80      2696



In [ ]:
# Anxiety BERT
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_all_anxiety.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      3108
           1       0.29      0.35      0.32       444

    accuracy                           0.81      3552
   macro avg       0.60      0.61      0.60      3552
weighted avg       0.83      0.81      0.82      3552



In [ ]:
# depression moe
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/moe_all_depression.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.49).astype(int)))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90      2359
           1       0.36      0.48      0.41       337

    accuracy                           0.83      2696
   macro avg       0.64      0.68      0.65      2696
weighted avg       0.85      0.83      0.84      2696



In [ ]:
# anxiety moe
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/moe_all_anxiety.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.56).astype(int)))

              precision    recall  f1-score   support

           0       0.91      0.85      0.88      3108
           1       0.27      0.38      0.32       444

    accuracy                           0.80      3552
   macro avg       0.59      0.62      0.60      3552
weighted avg       0.83      0.80      0.81      3552



In [ ]:
# depression ense
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/ens-concat_all_depression.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.76).astype(int)))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2359
           1       0.37      0.43      0.40       337

    accuracy                           0.84      2696
   macro avg       0.65      0.66      0.65      2696
weighted avg       0.85      0.84      0.84      2696



In [ ]:
# anxiety ense
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/ens-concat_all_anxiety.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.52).astype(int)))

              precision    recall  f1-score   support

           0       0.91      0.81      0.86      3108
           1       0.26      0.47      0.34       444

    accuracy                           0.77      3552
   macro avg       0.59      0.64      0.60      3552
weighted avg       0.83      0.77      0.80      3552



In [ ]:
# ansiedade usar 0.5
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_balanced_depression.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.96      0.47      0.63      2359
           1       0.19      0.87      0.31       337

    accuracy                           0.52      2696
   macro avg       0.58      0.67      0.47      2696
weighted avg       0.86      0.52      0.59      2696



In [ ]:
# ansiedade usar 0.5
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_balanced_anxiety.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.98      0.27      0.42      3108
           1       0.16      0.95      0.27       444

    accuracy                           0.36      3552
   macro avg       0.57      0.61      0.35      3552
weighted avg       0.87      0.36      0.41      3552



In [ ]:
# ansiedade usar 0.5
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/moe_all_depression.pkl', 'rb'))
print(classification_report(df['real'], (df['pred_all'] > 0.49).astype(int)))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90      2359
           1       0.36      0.48      0.41       337

    accuracy                           0.83      2696
   macro avg       0.64      0.68      0.65      2696
weighted avg       0.85      0.83      0.84      2696



In [ ]:
# ansiedade usar 0.5
df = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/all_depression.pkl', 'rb'))
print(classification_report(df['real'], df['pred']))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89      2359
           1       0.32      0.41      0.36       337

    accuracy                           0.82      2696
   macro avg       0.62      0.64      0.63      2696
weighted avg       0.84      0.82      0.83      2696



ENSEMBLE COM SELEÇÃO
##### seleciona 3 modelos de ensemble já treinados e decide se esse permanece ou não dependendo de quanto melhora o desempenho.

In [ ]:
import re 
import gc
import itertools

def make_prediction(model, dataloader_, model_name):
    model.eval()
    with torch.no_grad():
        # validation loop
        all_pred = []
        for _ in range(50):
            pred, real = [], []
            for j, batch in enumerate(dataloader_):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                
                if model_name == 'ens':
                    outputs, model_outputs = model(atts, masks)
                else:
                    outputs = model(atts, masks)
                # outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            print(classification_report(real, pred))
            all_pred.append(pred)

        p = (np.array(all_pred).mean(0) > 0.5).astype(int)
        # pred, real = [], []
        # for j, batch in enumerate(dataloader_):
        #     atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
        #     masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
        #     if model_name == 'ens':
        #         outputs, model_outputs = model(atts, masks)
        #     else:
        #         outputs = model(atts, masks)

        #     pred = pred + outputs.cpu().argmax(1).tolist()
        #     real = real + [b[1] for b in batch]
        print("*" * 20, '\nALL')
        print(classification_report(real, p))
        print("*" * 20)
        d = {
            'pred': p,
             'real': real,
             'pred_all': np.array(all_pred).mean(0)
        }
        return d


def evaluate_results(task, sel, model_name):
    gc.collect()
    device = 'cuda'
    if model_name == 'lstm':
        model = BertUser(device)
    elif model_name == 'moe':
        model = MoE(device)
    elif model_name == 'ens':
        model = Ensemble(device)
    elif model_name == 'ens-concat':
        model = EnsembleConcat(device)
    else:
        raise Exception('Model not defined')

    model.to(device)
    file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/'
    files = os.listdir(file_path)
    regex = re.compile(f'model-{model_name}-{task}-sel-{sel}-epoch')
    files = [f for f in files if regex.search(f)]
    
    # df_train, df_test = load_convert(task, sel)    
    tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
    # X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, sel)
    X_test, y_test = get_tokens_mask_dates_split(None, None, tokenizer, 'test', task, sel)    
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # del X_train
    # del y_train
    # del df_test
    gc.collect()
    # dataloader_val = get_dataloader(X_val, y_val, True)
    dataloader_test = get_dataloader(X_test, y_test, False)
    dict_prediction = {}
    files = [
             'model-lstm-A-sel-all-epoch7.pt',
             'model-lstm-A-sel-all-epoch11.pt',
             'model-lstm-A-sel-all-epoch12.pt',
    ]
    for f in files:
        print('*' * 20, f'\n{f}')
        try:
            load_checkpoint(f'{file_path}{f}', model)
        except:
            print('Could not load', f'{file_path}{f}')
            continue
        model.eval()
        d = make_prediction(model, dataloader_test, model_name)
        dict_prediction[f] = d
    d_p = {'A': 'anxiety', 'D': 'depression'}
    pickle.dump(dict_prediction, open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/{model_name}_{sel}_{d_p[task]}.pkl', 'wb'))
    # del df_train
    del X_test
    del y_test
    del dataloader_test
    gc.collect()



tasks = ['D']
selection = ['all']
models = ['lstm']

for task, sel, model_name in itertools.product(tasks, selection, models):
    print(task, sel, model_name)
    evaluate_results(task, sel, model_name)

D all lstm


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

load /content/drive/MyDrive/doutorado/diag_trat_saving/test_all_D_processed.pkl
******************** 
model-lstm-A-sel-all-epoch7.pt
Model loaded from <== /content/drive/MyDrive/doutorado/diag_trat_saving/all/model-lstm-A-sel-all-epoch7.pt
              precision    recall  f1-score   support

           0       0.93      0.57      0.70      2359
           1       0.19      0.69      0.29       337

    accuracy                           0.58      2696
   macro avg       0.56      0.63      0.50      2696
weighted avg       0.83      0.58      0.65      2696

              precision    recall  f1-score   support

           0       0.93      0.56      0.70      2359
           1       0.19      0.71      0.30       337

    accuracy                           0.58      2696
   macro avg       0.56      0.63      0.50      2696
weighted avg       0.84      0.58      0.65      2696

              precision    recall  f1-score   support

           0       0.93      0.56      0.70      23

In [ ]:
model_name = 'lstm'
task = 'A'
sel = 'all'
file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/'
files = os.listdir(file_path)
regex = re.compile(f'model-{model_name}-{task}-sel-{sel}-epoch')
files = [f for f in files if regex.search(f)]


In [ ]:
11, 12, 7, 2, 3, 5, 6 #D

['model-lstm-A-sel-all-epoch0.pt',
 'model-lstm-A-sel-all-epoch1.pt',
 'model-lstm-A-sel-all-epoch2.pt',
 'model-lstm-A-sel-all-epoch3.pt',
 'model-lstm-A-sel-all-epoch4.pt',
 'model-lstm-A-sel-all-epoch5.pt',
 'model-lstm-A-sel-all-epoch6.pt',
 'model-lstm-A-sel-all-epoch7.pt',
 'model-lstm-A-sel-all-epoch8.pt',
 'model-lstm-A-sel-all-epoch9.pt',
 'model-lstm-A-sel-all-epoch10.pt',
 'model-lstm-A-sel-all-epoch11.pt',
 'model-lstm-A-sel-all-epoch12.pt',
 'model-lstm-A-sel-all-epoch13.pt',
 'model-lstm-A-sel-all-epoch14.pt',
 'model-lstm-A-sel-all-epoch15.pt',
 'model-lstm-A-sel-all-epoch16.pt',
 'model-lstm-A-sel-all-epoch17.pt',
 'model-lstm-A-sel-all-epoch18.pt',
 'model-lstm-A-sel-all-epoch19.pt',
 'model-lstm-A-sel-all-epoch20.pt',
 'model-lstm-A-sel-all-epoch21.pt',
 'model-lstm-A-sel-all-epoch22.pt',
 'model-lstm-A-sel-all-epoch23.pt',
 'model-lstm-A-sel-all-epoch24.pt',
 'model-lstm-A-sel-all-epoch25.pt',
 'model-lstm-A-sel-all-epoch26.pt',
 'model-lstm-A-sel-all-epoch27.pt',
 '

In [ ]:
dict_prediction = pickle.load(open(f'/content/drive/MyDrive/doutorado/dados_v6/full/prediction/lstm_all_anxiety.pkl', 'rb'))

FileNotFoundError: ignored

In [ ]:
pred = []
for k in dict_prediction:
    pred.append((np.array(dict_prediction[k]['pred_all']) > 0.6).astype(int))

In [ ]:
p = (np.array(pred).mean(0) == 1).astype(int)

In [ ]:
(np.array(dict_prediction[k]['pred_all']) > 0.4).astype(int)

array([0, 1, 0, ..., 0, 1, 1])

In [ ]:
print(classification_report(dict_prediction[k]['real'], p))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      2359
           1       0.33      0.31      0.32       337

    accuracy                           0.84      2696
   macro avg       0.62      0.61      0.61      2696
weighted avg       0.83      0.84      0.83      2696

